# Modelo Gurobi - Equipo 72
Gestión de Relaves Abandonados e Inactivos en la Comuna de Copiapó: Maximización de Beneficios Sociales sujeto a Costos y Ponderadores de Riesgos Socioambientales

Imports iniciales a usar en el resto del código:

In [2]:
from gurobipy import Model, GRB, quicksum
from collections import defaultdict
from shapely.geometry import Point
from random import randint
import geopandas as gpd
import numpy as np
import itertools
import time
import os
from funciones import *

Primero, definimos todos los paths de los archivos a usar.

In [5]:
DIR_PATH = os.getcwd()
DATA_PATH = os.path.join(DIR_PATH, 'data')

COMUNA_PATH = os.path.join(DATA_PATH, 'division-territorial', 'division-territorial.shp')
URBANO_PATH = os.path.join(DATA_PATH, 'zona-urbana', 'zona-urbana.shp')
PARQUE_PATH = os.path.join(DATA_PATH, 'zona-urbana-parque', 'zona-urbana-parque.shp')
DEPOSITOS_PATH = os.path.join(DATA_PATH, 'depositos.csv')
AGUA_PATH = os.path.join(DATA_PATH, 'fuentes-agua', 'fuentes-agua.shp')

PUNTOS_PATH = os.path.join(DATA_PATH, 'puntos-grid', 'conjunto-puntos.csv')
PUNTOS_SOL_PATH = os.path.join(DATA_PATH, 'puntos-solucion', 'conjunto-puntos.csv')

VECTOR_PATH = os.path.join(DATA_PATH, 'vector-layer', 'vector-layer.shp')
VECTOR_SOL_PATH = os.path.join(DATA_PATH, 'vector-layer-sol', 'vector-layer.gpkg')

### Parte 1: Conjunto de posibles posiciones de depósitos de relave, zonas urbanas, fuentes de agua.
Para crear este conjunto utilizamos Shapefiles de [Geoportal de Chile](geoportal.cl) incluyendo la división político administrativa, la planificación urbana dentro de la comuna de Copiapó, y el conjunto de todos los depósitos de relaves. El parámetro usado en el código llamado "GRID_TILES_AVG" permite calcular la distancia entre los puntos, que es el promedio de ambos lados que encierran la comuna, partido por GRID_TILES_AVG. Mientras más alto es este número, más puntos posibles para los relaves tenemos con datos más cercanos a la realidad. \

Sin embargo, incrementar mucho este parámetro hace que el modelo de Gurobi se demore mucho.

In [4]:
GRID_TILES_AVG = 10

In [6]:
print("Cargando las posiciones de los depósitos antiguos")
D = []
with open(DEPOSITOS_PATH, 'r', encoding='utf-8') as archivo:
    _ = next(archivo)  # sacar primera linea
    for linea in archivo:
        coords = linea.strip().split(",")[20:22][::-1]
        punto = Point(float(coords[0]), float(coords[1]))
        D.append(punto)
print("Done!")

# conjunto de todas las posiciones, zonas urbanas y fuentes de agua
print("Cargando todos los conjuntos a partir de shapefiles")
comienzo = time.time()
comuna_shape = gpd.read_file(COMUNA_PATH)
urbano_shape = gpd.read_file(URBANO_PATH)
parque_shapes = gpd.read_file(PARQUE_PATH)
agua_shapes = gpd.read_file(AGUA_PATH)
urbano_shape = urbano_shape.to_crs(comuna_shape.crs)  # mismo coordinate system

bounds = comuna_shape.total_bounds
min_x, min_y, max_x, max_y = bounds

dist_x = max_x - min_x
dist_y = max_y - min_y
distance = (dist_x / GRID_TILES_AVG + dist_y / GRID_TILES_AVG) / 2

P = [*D]
U = []
A = []

# comenzar desde abajo a la izquierda pero no en el borde
current_point = Point(min_x + distance, min_y + distance)

i = 0
total_i = dist_y // distance - 1
while min_x < current_point.x < max_x and min_y < current_point.y < max_y:
    if comuna_shape.contains(current_point).bool():
        P.append(current_point)
    if (urbano_shape.contains(current_point).bool() or
            any([parque.contains(current_point) for parque in parque_shapes.geometry])):
        U.append(current_point)
    if any([agua.contains(current_point) for agua in agua_shapes.geometry]):
        A.append(current_point)

    new_x = current_point.x + distance
    new_y = current_point.y
    if new_x > max_x:
        print(f"{round(i / total_i * 100, 2)}%")
        new_x = min_x + distance
        new_y = current_point.y + distance
        i += 1
    current_point = Point(new_x, new_y)

print(f"Done in {round(time.time() - comienzo, 1)}s!")

print(f"Cantidad de puntos: {len(P)}")
print(f"Cantidad de zonas urbanas: {len(U)}")
print(f"Cantidad de fuentes de agua: {len(A)}")

print("Escribiendo en archivo...")
comienzo = time.time()
with open(PUNTOS_PATH, 'w', encoding='utf-8') as archivo:
    print("longitud,latitud,tipo", file=archivo)  # header

    for i, punto in enumerate(P):
        if i % 1000 == 0:
            print(f"{i} puntos escritos...")
        tipo = 'P'
        if punto in U:
            tipo += ';U'
        if punto in A:
            tipo += ';A'
        if punto in D:
            tipo += ';D'
        print(f'{punto.x},{punto.y},{tipo}', file=archivo)
print(f"Done in {round(time.time() - comienzo, 1)}s")

Cargando las posiciones de los depósitos antiguos
Done!
Cargando todos los conjuntos a partir de shapefiles
0.0%
20.0%
40.0%
60.0%
80.0%
100.0%
Done in 1.6s!
Cantidad de puntos: 112
Cantidad de zonas urbanas: 4
Cantidad de fuentes de agua: 2
Escribiendo en archivo...
0 puntos escritos...
Done in 0.0s


C:\Users\pedro\AppData\Local\Temp\ipykernel_13328\1798487054.py:37: FutureWarning: Series.bool is now deprecated and will be removed in future version of pandas
  if comuna_shape.contains(current_point).bool():
C:\Users\pedro\AppData\Local\Temp\ipykernel_13328\1798487054.py:39: FutureWarning: Series.bool is now deprecated and will be removed in future version of pandas
  if (urbano_shape.contains(current_point).bool() or


Este archivo se guarda en `/data/puntos-grid/conjunto-puntos.csv` para su lectura más adelante.

### Parte 2: Modelo de Gurobi

In [7]:
# ------- iniciar modelo -------
inicio = time.time()
md = Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2026-09-11


In [8]:
# -------   conjuntos    -------
puntos = []
P, U, D, A = ([] for _ in range(4))
with open(PUNTOS_PATH, 'r', encoding='utf-8') as archivo:
    _ = next(archivo)
    for i, linea in enumerate(archivo):
        punto = linea.strip().split(',')
        punto[0], punto[1] = float(punto[0]), float(punto[1])
        punto[2] = punto[2].split(';')
        punto_coords = Point(punto[:2])
        puntos.append(punto_coords)
        for tipo in punto[2]:
            if tipo in 'PUDA':
                [P, U, D, A]['PUDA'.index(tipo)].append(i)

"""
Los conjuntos P, U, D, A son:
P: conjunto de posibles de depósitos de relave, zonas urbanas, entre otros.
U: subconjunto de zonas urbanas.
D: subconjunto de posiciones de depósitos de relaves iniciales.
A: subconjunto de posiciones de fuentes de agua
F: conjunto de estrategias de fitorremediación aplicables
Todos estos conjuntos trabajan sobre los INDICES de sus coordenadas respectivas en la lista "puntos"
"""
# 0: fitoextraccion, 1: fitoestabilizacion, 2: rizorremediacion
F = [i for i in range(3)]

In [9]:
# -------   parametros   -------
PS = 50_000_000**2
M = PS
T = np.array([randint(27, 21_244_362) for _ in P])
CT = np.array([randint(200, 1000) for _ in P])
CF = [
    [randint(5, 50) for _ in P],
    [randint(25, 200) for _ in P],
    [randint(200, 800) for _ in P],
]
CS = np.array([randint(10_000, 100_000_000) for _ in P])
# para ahorrar runtime, L va a ser una función que calcula la distancia entre dos puntos.
L = lambda p, pp: haversine(puntos[p], puntos[pp])
KI = leer_cantidades_iniciales(DEPOSITOS_PATH) + [0 for _ in range(len(P) - len(D))]  # 0 tons para cada posición nueva
MCNTU = {u: randint(90, 100) for u in U}
MCNTA = {a: randint(50, 90) for a in A}
MCNTU = {u: 100_000 for u in U}  # fix
MCNTA = {a: 100_000 for a in A}  # fix
CNT = 0.02
ALPHA = 0.2  # 20% de la cantidad percibida original
BETA = np.array([0.2, 0.6, 0.3])
DM = 1.5  # en kilometros, distancia entre relaves
DMU = 10  # NEW, en kilometros, distancia entre relave-ciudad
# conversion: 1.5 km = 0.02126° (latitud, longitud) ; 1° = 70.5550329 km
LAMBDA = np.array([0.35, 0.35, 0.30])

In [11]:
# -------   variables    -------
print("Creando variables...")

p_pp = list(itertools.product(P, P))
pf = list(itertools.product(P, F))

XT = md.addVars(p_pp, vtype=GRB.BINARY, name="XT")
XTR = md.addVars(P, vtype=GRB.BINARY, name="XTR")
XF = md.addVars(pf, vtype=GRB.BINARY, name="XF")
XS = md.addVars(P, vtype=GRB.BINARY, name="XS")
W = md.addVars(p_pp, vtype=GRB.CONTINUOUS, lb=0, name="W")
K = md.addVars(P, vtype=GRB.CONTINUOUS, lb=0, name="K")
C = md.addVars(P, lb=-GRB.INFINITY, ub=GRB.INFINITY, name="C")
Z = md.addVars(P, vtype=GRB.CONTINUOUS, lb=0, name="Z")
ZV = md.addVars(P, vtype=GRB.CONTINUOUS, lb=0, name="ZV")
KA = md.addVars(pf, vtype=GRB.CONTINUOUS, lb=0, name="KA")
Y = md.addVars(P, vtype=GRB.BINARY, name="Y")  # NEW, HAY RELAVE EN LA POSICIÓN P

md.update()
print("Variables creadas")

Creando variables...
Variables creadas


In [12]:
# -------  restricciones -------
print("Aplicando restricciones...")

# restricciones de variables
print("- Restricciones de variables")
md.addConstrs(XTR[p] + quicksum(XF[p, f] for f in F) + XS[p] <= 1 for p in D)
md.addConstrs(XTR[p] + quicksum(XF[p, f] for f in F) + XS[p] == 0 for p in P if p not in D)
md.addConstrs(quicksum(XT[p, pp] for pp in P) <= M * XTR[p] for p in P)
md.addConstrs(W[p, pp] <= M * XT[p, pp] for p in P for pp in P if p != pp)
md.addConstrs(M * (1 - (XTR[p] + quicksum(XF[p, f] for f in F) + XS[p])) >= quicksum(XT[pp, p] for pp in P) for p in P)
md.addConstrs(M * Y[p] >= quicksum(XT[p, pp] for pp in P) for p in P)  # NEW
md.addConstrs(M * Y[p] >= 1 - XTR[p] for p in D)  # NEW, si no se ha transladado nada en d entonces es un relave (Y = 1)
md.addConstrs(quicksum(XT[p, pp] for pp in P) <= 1 for p in P)  # NEW, se puede transladar a lo más a 1 lugar
print("- Restricciones de variables: Done")

# restricciones de presupuesto
print("- Restricciones de presupuesto")
md.addConstrs(KA[p, f] <= M * XF[p, f] for p in D for f in F)
md.addConstrs(KA[p, f] <= K[p] for p in D for f in F)
md.addConstrs(KA[p, f] >= K[p] - M * (1 - XF[p, f]) for p in D for f in F)
md.addConstr(quicksum(quicksum(W[p, pp] * L(p, pp) for pp in P) * CT[p] + quicksum(KA[p, f] * CF[f][p] for f in F) + XS[p] * CS[p] for p in P) <= PS)
md.addConstrs(K[p] == KI[p] + quicksum(W[i, p] for i in P) - quicksum(W[p, j] for j in P) for p in P)
md.addConstrs(K[p] <= T[p] for p in P)
print("- Restricciones de presupuesto: Done")

# restricciones de contaminación
print("- Restricciones de contaminación")
md.addConstrs(Z[p] == K[p] * CNT for p in P)
md.addConstrs(ZV[p] - ALPHA * Z[p] <= M * (1 - XS[p]) for p in P)
md.addConstrs(ZV[p] - ALPHA * Z[p] >= -M * (1 - XS[p]) for p in P)
md.addConstrs(ZV[p] - BETA[f] * Z[p] <= M * (1 - XF[p, f]) for p in P for f in F)
md.addConstrs(ZV[p] - BETA[f] * Z[p] >= -M * (1 - XF[p, f]) for p in P for f in F)
md.addConstrs(ZV[p] - Z[p] <= M * (quicksum(XF[p, f] for f in F) + XS[p]) for p in P)
md.addConstrs(ZV[p] - Z[p] >= -M * (quicksum(XF[p, f] for f in F) + XS[p]) for p in P)
md.addConstrs(quicksum(ZV[p] / L(p, u) for p in P if p != u) <= MCNTU[u] for u in U)
md.addConstrs(quicksum(ZV[p] / L(p, a) for p in P if p != a) <= MCNTA[a] for a in A)
md.addConstrs(MCNTU[u] - quicksum(ZV[p] / L(p, u) for p in P if p != u) == C[u] for u in U)
md.addConstrs(MCNTA[a] - quicksum(ZV[p] / L(p, a) for p in P if p != a) == C[a] for a in A)
md.addConstrs(C[p] == 0 for p in P if p not in (U + A))
print("- Restricciones de contaminación: Done")

# restricciones de distancia
print("- Restricciones de distancia")
md.addConstrs(L(p, pp) + M * (1 - Y[p]) >= DM for p in P for pp in P if p != pp)  # NEW
md.addConstrs(L(p, u) + M * (1 - Y[p]) >= DMU for p in P for u in U)  # NEW
md.addConstrs(K[u] == 0 for u in U)
print("- Restricciones de distancia: Done")


md.update()

Aplicando restricciones...
- Restricciones de variables
- Restricciones de variables: Done
- Restricciones de presupuesto
- Restricciones de presupuesto: Done
- Restricciones de contaminación
- Restricciones de contaminación: Done
- Restricciones de distancia
- Restricciones de distancia: Done


In [13]:
# -------  función obj  -------
print("Agregando función objetivo...")
md.setObjective(quicksum(LAMBDA[0] * quicksum(XF[p, f] for f in F) + LAMBDA[1] * XS[p] + LAMBDA[2] * C[p] for p in P), GRB.MAXIMIZE)
print("Optimizando...")

md.optimize()
print("Optimizado")

Agregando función objetivo...
Optimizando...
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: AMD Ryzen 5 3550H with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 28222 rows, 53088 columns and 134708 nonzeros
Model fingerprint: 0x22eac728
Variable types: 26656 continuous, 26432 integer (26432 binary)
Coefficient statistics:
  Matrix range     [4e-03, 3e+15]
  Objective range  [3e-01, 3e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+15]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 27087 rows and 51408 columns
Presolve time: 0.77s
Presolved: 1135 rows, 1680 columns, 5672 nonzeros
Variable types: 851 continuous, 829 integer (828 binary)
Found heuristic solution: objective 137423.27850
Found heuristic solution: objective 137606.19846

Root relaxation: objective 1

#### Resultados

In [22]:
valor_objetivo = md.ObjVal
tiempo_ejecucion = md.Runtime

print("-"*50)
print(f"El beneficio social alcanzado fue de {round(valor_objetivo)}")
print("-"*50)


for p in P:

    hizo_algo = False
    if K[p].x != 0:
        print("-"*50)
        print(f"En la posición {p} hay {K[p].x} toneladas de relave")

    for pp in P:
        if XT[p, pp].x == 1:
            hizo_algo = True
            print("-"*50)
            print(f"El depósito en la posición {p} se movió a {pp}")

    for f in F:
        if XF[p, f].x == 1:
            hizo_algo = True
            print("-"*50)
            print(f"Se aplicó el método {f} al depósito de relave en la posición {p}")

    if XS[p].x == 1:
        hizo_algo = True
        print("-"*50)
        print(f"Se selló el depósito en {p}")
        print(f"Tiene {K[p].x} toneladas")

    if not hizo_algo and p in D:
        print(f"La posición {p} se mantuvo en el mismo lugar")
        print("-"*50)


--------------------------------------------------
El beneficio social alcanzado fue de 140402
--------------------------------------------------
--------------------------------------------------
Se selló el depósito en 0
Tiene 0.0 toneladas
--------------------------------------------------
Se aplicó el método 1 al depósito de relave en la posición 1
--------------------------------------------------
Se selló el depósito en 2
Tiene 0.0 toneladas
--------------------------------------------------
Se aplicó el método 1 al depósito de relave en la posición 3
--------------------------------------------------
Se aplicó el método 1 al depósito de relave en la posición 4
--------------------------------------------------
En la posición 5 hay 340897.0 toneladas de relave
--------------------------------------------------
Se aplicó el método 0 al depósito de relave en la posición 5
--------------------------------------------------
En la posición 6 hay 132468.0 toneladas de relave
----------